In [1]:
import json
import os
import numpy as np
import sys
import tensorflow as tf
from tqdm import tqdm_notebook

from src import model, sample, encoder

W0809 10:15:34.355935 13140 deprecation_wrapper.py:119] From C:\Users\AI-Machine\Documents\gpt-2\src\model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.



In [2]:

def generateMoviePlots(
    movieSummaries,
    model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    top_p=0.0
):
    """
    Interactively run the model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
    :top_p=0.0 : Float value controlling diversity. Implements nucleus sampling,
     overriding top_k if set to a value > 0. A good setting is 0.9.
    """
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name)
    hparams = model.default_hparams()
    with open(os.path.join('models', model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k, top_p=top_p
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
        saver.restore(sess, ckpt)

        with open("generated_movie_plots.txt", "w", encoding="utf-8") as f:
            # make a loop here over the movie-plots
            for i in tqdm_notebook(range(0, len(movieSummaries), 2)):
                # get the summary as input
                raw_text = "[WP] " + movieSummaries[i+1].strip()
                # write the name of the movie followed by the generated plot in a file
                f.write("[Movie] " + movieSummaries[i].strip() + "\n")
                f.write("[Plot] " + movieSummaries[i+1].strip() + "\n")
                print("======================================== INPUT ========================================\n", raw_text)
                while not raw_text:
                    print('Prompt should not be empty!')
                    raw_text = input("Model prompt >>> ")
                context_tokens = enc.encode(raw_text)
                generated = 0
                for _ in range(nsamples // batch_size):
                    out = sess.run(output, feed_dict={
                        context: [context_tokens for _ in range(batch_size)]
                    })[:, len(context_tokens):]
                    for i in range(batch_size):
                        generated += 1
                        text = enc.decode(out[i])
                        f.write(text+"\n")
                        print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                        print(text)
        print("=" * 80)

        
with open("data/movie_plot_data.txt", "r", encoding="utf-8") as f:
    movie_plots = f.readlines()
len(movie_plots)
generateMoviePlots(movie_plots, model_name="writingprompts117M", top_p=0.9, nsamples=1, length=None)

W0809 10:15:37.620572 13140 deprecation_wrapper.py:119] From C:\Users\AI-Machine\Documents\gpt-2\src\model.py:148: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0809 10:15:37.632566 13140 deprecation_wrapper.py:119] From C:\Users\AI-Machine\Documents\gpt-2\src\model.py:152: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0809 10:15:37.680563 13140 deprecation_wrapper.py:119] From C:\Users\AI-Machine\Documents\gpt-2\src\model.py:36: The name tf.rsqrt is deprecated. Please use tf.math.rsqrt instead.

W0809 10:15:39.200562 13140 deprecation_wrapper.py:119] From C:\Users\AI-Machine\Documents\gpt-2\src\model.py:166: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

W0809 10:15:40.385595 13140 deprecation.py:323] From C:\Users\AI-Machine\Documents\gpt-2\src\sample.py:65: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a futur

======================================== INPUT ========================================
 [WP] Chronicles the experiences of a formerly successful banker as a prisoner in the gloomy jailhouse of Shawshank after being found guilty of a crime he did not commit. The film portrays the man's unique way of dealing with his new, torturous life; along the way he befriends a number of fellow prisoners, most notably a wise long-term inmate named Red.
======================================== SAMPLE 1 ========================================
 
Henry shouted as he struggled to carry his grief towards the green tiling tile in his cell. He looked around and the crowd sat at attention to notice that a fourth group of guards was holding a sign that read 'Dying Prisoner'. They were bent over a chair in separate cells. For the first time in three weeks, he paced in front of a screen of white, glittering doors and packed the space he had for a meeting. He watched the entire prison come down around him, eac

 [WP] Set within a year after the events of Batman Begins, Batman, Lieutenant James Gordon, and new district attorney Harvey Dent successfully begin to round up the criminals that plague Gotham City until a mysterious and sadistic criminal mastermind known only as the Joker appears in Gotham, creating a new wave of chaos. Batman's struggle against the Joker becomes deeply personal, forcing him to "confront everything he believes" and improve his technology to stop him. A love triangle develops between Bruce Wayne, Dent and Rachel Dawes.
======================================== SAMPLE 1 ========================================
 <|endoftext|>
[WP] You’re a power hungry teenager who hates pizza. So you decide to make the world a better place by making a killer pizza. 
"I'm sorry, but your pizza comes without wiggles or toppings." The man pointed at the secret ingredient in a musty dough pizza, “Jesus Christ. That was me. I can’t believe I did it. I ruined it!" Baumbach rolled his eyes. A 

[WP] In another country, the Jews are getting their exterminations done so well that there is barely any visible resistance to the extermination process. But in this case, it was an ordinary Jewish worker who does the job. Oskar Schindler. One of Germany's most notorious criminals. Back in his old country, the bravest of the worst criminals – he had, after all, saved the innocents in France and Germany – sorted them out on the plane and unloaded them onto the North Sea. Oskar agreed, believing that Germany and its allies deserved the success it achieved. With his parents and grandmother, he had a job doing 'em. When the Nazis came to power, Oskar volunteered. He had proved himself almost as good as they were. The recruits were still coming up, but he had done more than anyone else to win the hearts of the German soldiers and the delegates
======================================== INPUT ========================================
 [WP] suitcase stolen from their employer, mob boss Marsellus

The grass was never grey as my family moved into this place. It was new and the land was new, but I had never seen it. The grass is so dull and dull. As the years had passed, the grass grew. Because there were new trees and new grass and new flowers. My wife and I had been running our garden down to the trees and the grass, but we were not allowed to make new or repurpose the old tree here. I heard from the pastor I got to go to to give thanks and cheer up his flock. I waited for the preacher who I would get to see, and the young lady who I would go to to pray. I was beaming with joy when she saw a scarred thing lying on the ground that I could not see anymore. It looked like a human hand. That is what a human's hand looks like. I moved the hairs on my body to avoid it and took a sharp angle towards the thing. The thing took a long time to tear itself free from the vines, but the moisture was kind enough to leave a trail that I could take in my head. "Look at that!" I heard the red of 

...

 [WP] An unknown and lethal virus has wiped out five billion people in 1996. Only 1% of the population has survived by the year 2035, and is forced to live underground. A convict (James Cole) reluctantly volunteers to be sent back in time to 1996 to gather information about the origin of the epidemic (who he's told was spread by a mysterious "Army of the Twelve Monkeys") and locate the virus before it mutates so that scientists can study it. Unfortunately Cole is mistakenly sent to 1990, six years earlier than expected, and is arrested and locked up in a mental institution, where he meets Dr. Kathryn Railly, a psychiatrist, and Jeffrey Goines, the insane son of a famous scientist and virus expert.
======================================== SAMPLE 1 ========================================
 
He had just come into the room. He thought of the horrors of being put in that environment with a 20 year old guy who was alone and inexperienced in the field. A vortex. A blizzard. A cancer. A world 

The top was a wasteland. Terribly irradiated. Burnished skyscrapers snapped by an idea. Hard to argue. It wasn't really the fault of the agency, who got it right and the way it was handled, that I was responsible for littering the baddest of my various pieces of garbage and slowly destroying a burgeoning government agency that I'd long had an issue with. Perhaps someone will be right, because they've been messing with me all day, and I was getting sidetracked by a gigantic argument about a set of rules that apply to me all over the world. The first one: I was to have a child. To be a nanny. Not in the grand scheme of things, a lovely lady running a desk job for the person that would then act as their super bank teller for the couple's business, but that was the whole point. I was not a perfect husband or a good mother, but I did care for them and I carried on as my wife should. But I did care for the babies of the agency, and I had one last child for a while. And then I didn't come in 

 [WP] Painfully shy Todd Anderson has been sent to the school where his popular older brother was valedictorian. His roommate, Neil Perry, although exceedingly bright and popular, is very much under the thumb of his overbearing father. The two, along with their other friends, meet Professor Keating, their new English teacher, who tells them of the Dead Poets Society, and encourages them to go against the status quo. Each does so in his own way, and is changed for life.
======================================== SAMPLE 1 ========================================
 
Quiet and calm there. You had read that line a thousand times, before you calmed yourself down. Despite the cacophony of irritation you felt while hesitating, you knew you were very, very close. <|endoftext|>
[WP] A brief period after the end of the fourth world war, the Republic is banned from becoming a military force. You are, however, gifted the ability to temporarily disable the effects of a weapon and damage it. 
"Ah, every

[WP] Based upon a real-life story that happened in the early seventies in which the Chase Manhattan Bank in Gravesend, Brooklyn, was held siege by a bank robber determined to steal enough money for his male lover to undergo a sex change operation. On a hot summer afternoon, the First Savings Bank of Brooklyn is held up by Sonny and Sal, two down-and-out characters. Although the bank manager and female tellers agree not to interfere with the robbery, Sonny finds that there's actually nothing much to steal, as most of the cash has been picked up for the day. Sonny then gets an unexpected phone call from Police Captain Moretti, who tells him the place is surrounded by the city's entire police force. Having few options under the circumstances, Sonny reluctantly bargains with Moretti, demanding safe escort to the airport and a plane out of the country in return for the bank employees' safety. The undercover police officer cajoles the hostage into signing off with more of a 'But I swear, and

My heart pounded like a boat through my chest as she had last night; her first high. I was overcome with shame when the news came. I would have dropped everything all at once, grabbed my two workhorses, walked on top of it, bolted to the bathroom, and strolled up to bed. I didn't know what had hit me, but I knew my job. I was half way across the country when the door to my boss' office opened. I was met with a group of people I didn't recognize, who asked me to take a tour of the office. I hadn't been able to make it all the way to the desk, so I walked past them. I was struck by how much excitement I was feeling. The secretary was walking up to my desk. After a moment I was reminded of what the CEO said; "I've asked you to be an ambassador." I couldn't make out what she was saying, but every word was rough, so I nodded. The guy didn't say anything except just nod and nod back. "I see you are an ambassador. Do you believe in telling the truth? Do you believe it is better to lie? Or is 